# Lottery Ticket Hypothesis

In [1]:
#Imports
import numpy
import tensorflow
from sklearn import preprocessing, model_selection
from lottery_ticket_hypothesis import PrunableDense
from tensorflow.keras import models, layers, activations
from tensorflow import optimizers, initializers, losses, metrics

In [2]:
#Tries to enable dynamic memory allocation on GPUs
try:
    for i in tensorflow.config.experimental.list_physical_devices("GPU"):
        tensorflow.config.experimental.set_memory_growth(i, True)
except:
    print("Dynamic memory allocation failed on GPU device!")

In [3]:
def create_neural_network_prunable():
    """Prunable model of a fully-connected multilayer perceptron"""
    net = models.Sequential()
    net.add(PrunableDense(256, activation=activations.softsign, name="Dense0", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(128, activation=activations.softsign, name="Dense1", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(64, activation=activations.softsign, name="Dense2", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(32, activation=activations.softsign, name="Dense3", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    net.add(PrunableDense(1, activation=activations.tanh, name="Output", bias_initializer=tensorflow.ones, kernel_initializer=initializers.he_normal()))
    return net

In [4]:
#Data reading and train/test separation
X = numpy.loadtxt("poco_1.prn", skiprows=11, usecols=(1, 2, 3, 4, 5, 6, 7), dtype=numpy.float32)
y_str = numpy.loadtxt("poco_1.prn", skiprows=11, usecols=8, dtype=numpy.str)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(list(set(y_str)))
y = label_encoder.transform(y_str)
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
#Building model
print("Prunable network:")
nn = create_neural_network_prunable()
nn.build(x_train.shape)
nn.summary()

Prunable network:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense0 (PrunableDense)       multiple                  2048      
_________________________________________________________________
Dense1 (PrunableDense)       multiple                  32896     
_________________________________________________________________
Dense2 (PrunableDense)       multiple                  8256      
_________________________________________________________________
Dense3 (PrunableDense)       multiple                  2080      
_________________________________________________________________
Output (PrunableDense)       multiple                  33        
Total params: 45,313
Trainable params: 45,313
Non-trainable params: 0
_________________________________________________________________


In [6]:
#Saving initial values
for i in nn.layers:
    i.save_kernel()
    i.save_bias()

In [7]:
#compiling, training and evaluating model
nn.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
print("Before pruning:")
nn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_train, y_train))
loss, accuracy = nn.evaluate(x_test, y_test, verbose=0)
print("Loss:", loss)
print("Accuracy:", accuracy)

Before pruning:
Train on 1134 samples, validate on 1134 samples
Epoch 1/10
1134/1134 [==============================] - 2s 1ms/sample - loss: 0.7922 - binary_accuracy: 0.4612 - val_loss: 0.5773 - val_binary_accuracy: 0.6684
Epoch 2/10
1134/1134 [==============================] - 0s 90us/sample - loss: 0.5043 - binary_accuracy: 0.7143 - val_loss: 0.4227 - val_binary_accuracy: 0.8254
Epoch 3/10
1134/1134 [==============================] - 0s 91us/sample - loss: 0.3557 - binary_accuracy: 0.8730 - val_loss: 0.3010 - val_binary_accuracy: 0.9021
Epoch 4/10
1134/1134 [==============================] - 0s 98us/sample - loss: 0.3126 - binary_accuracy: 0.9083 - val_loss: 0.2773 - val_binary_accuracy: 0.9162
Epoch 5/10
1134/1134 [==============================] - 0s 92us/sample - loss: 0.2840 - binary_accuracy: 0.9153 - val_loss: 0.2915 - val_binary_accuracy: 0.9127
Epoch 6/10
1134/1134 [==============================] - 0s 91us/sample - loss: 0.3006 - binary_accuracy: 0.9145 - val_loss: 0.2849 -

In [8]:
l1 = []
for layer in nn.layers:
    l1.append(tensorflow.reshape(layer.kernel, (-1,))[tensorflow.reshape(layer.trainable_channels, (-1,)) == 1])
    l1.append(tensorflow.reshape(layer.bias, (-1,))[tensorflow.reshape(layer.trainable_bias, (-1,)) == 1])

In [9]:
s = tensorflow.sort(tensorflow.concat(l1, axis=-1))
p = int(numpy.floor((9. / 10.) * len(s)))
threshold = s[p].numpy()

In [10]:
for layer in nn.layers:
    indices_kernel = tensorflow.where(layer.kernel < threshold)
    indices_bias = tensorflow.where(layer.bias < threshold)
    t1 = tensorflow.tensor_scatter_nd_update(tensorflow.ones(layer.kernel.shape), indices_kernel, tensorflow.zeros(len(indices_kernel)))
    t2 = tensorflow.tensor_scatter_nd_update(tensorflow.ones(layer.bias.shape), indices_bias, tensorflow.zeros(len(indices_bias)))
    layer.prune_kernel(t1)
    layer.prune_bias(t2)

In [11]:
#Restoring initial values
for i in nn.layers:
    i.restore_kernel()
    i.restore_bias()

In [12]:
#Training again
print("After pruning:")
nn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_train, y_train))
loss, accuracy = nn.evaluate(x_test, y_test, verbose=0)
print("Loss:", loss)
print("Accuracy:", accuracy)

After pruning:
Train on 1134 samples, validate on 1134 samples
Epoch 1/10
1134/1134 [==============================] - 0s 106us/sample - loss: 0.9295 - binary_accuracy: 0.3369 - val_loss: 0.7372 - val_binary_accuracy: 0.3369
Epoch 2/10
1134/1134 [==============================] - 0s 91us/sample - loss: 0.6407 - binary_accuracy: 0.6526 - val_loss: 0.5695 - val_binary_accuracy: 0.7046
Epoch 3/10
1134/1134 [==============================] - 0s 93us/sample - loss: 0.5305 - binary_accuracy: 0.6808 - val_loss: 0.4824 - val_binary_accuracy: 0.7734
Epoch 4/10
1134/1134 [==============================] - 0s 92us/sample - loss: 0.4329 - binary_accuracy: 0.8201 - val_loss: 0.3745 - val_binary_accuracy: 0.8598
Epoch 5/10
1134/1134 [==============================] - 0s 94us/sample - loss: 0.3438 - binary_accuracy: 0.8889 - val_loss: 0.3114 - val_binary_accuracy: 0.8968
Epoch 6/10
1134/1134 [==============================] - 0s 95us/sample - loss: 0.2957 - binary_accuracy: 0.9048 - val_loss: 0.2982 

In [13]:
#Printing active edges
print("Active edges:")
t = tensorflow.Variable(0, dtype=tensorflow.int64)
for i in nn.layers:
    j = tensorflow.math.count_nonzero(i.trainable_channels)
    t.assign_add(j)
    print(i.name, ":", j)
print("Total:", t.numpy())

Active edges:
Dense0 : tf.Tensor(728, shape=(), dtype=int64)
Dense1 : tf.Tensor(1723, shape=(), dtype=int64)
Dense2 : tf.Tensor(1123, shape=(), dtype=int64)
Dense3 : tf.Tensor(465, shape=(), dtype=int64)
Output : tf.Tensor(12, shape=(), dtype=int64)
Total: 4051


In [14]:
t.assign(0)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=0>

In [15]:
print("Disabled edges:")
for i in nn.layers:
    j = tensorflow.reduce_sum(tensorflow.cast((i.trainable_channels == 0), dtype=tensorflow.int64))
    t.assign_add(j)
    print(i.name, ":", j)
print("Total:", t.numpy())

Disabled edges:
Dense0 : tf.Tensor(1064, shape=(), dtype=int64)
Dense1 : tf.Tensor(31045, shape=(), dtype=int64)
Dense2 : tf.Tensor(7069, shape=(), dtype=int64)
Dense3 : tf.Tensor(1583, shape=(), dtype=int64)
Output : tf.Tensor(20, shape=(), dtype=int64)
Total: 40781
